In [1]:
#保存浏览器驱动
from DrissionPage import ChromiumOptions
path = r'C:\Program Files\Google\Chrome\Application\chrome.exe'
ChromiumOptions().set_browser_path(path).save()

配置已保存到文件: Z:\anaconda3\envs\Django\lib\site-packages\DrissionPage\_configs\configs.ini
以后程序可自动从文件加载配置


'Z:\\anaconda3\\envs\\Django\\lib\\site-packages\\DrissionPage\\_configs\\configs.ini'

In [1]:
import pandas as pd

# 读取CSV文件
df = pd.read_csv('csv/RAM_products.csv')

# 1. 清洗评论数 (comment_count)
if 'comment_count' in df.columns:
    # 处理"万+"情况
    df['comment_count'] = df['comment_count'].astype(str).str.replace('万\+', '0000', regex=True)
    # 处理"+"情况
    df['comment_count'] = df['comment_count'].astype(str).str.replace('\+', '', regex=True)
    # 转换为数值类型
    df['comment_count'] = pd.to_numeric(df['comment_count'], errors='coerce')
else:
    print("警告：数据框中没有 'comment_count' 列")

# 2. 清洗价格数据 (保留你原有的价格清洗逻辑)
columns_to_clean = ['current_price', 'reference_price']
for column in columns_to_clean:
    if column in df.columns:
        # 去掉 '￥' 和 '¥' 符号
        df[column] = df[column].astype(str).str.replace('￥', '').str.replace('¥', '')

        # 处理 '万' 字，并转换为数值
        has_wan = df[column].str.contains('万', na=False)
        df.loc[has_wan, column] = (
            df.loc[has_wan, column]
            .str.replace('万', '')
            .astype(float) * 10000
        )

        # 确保所有值转换为数值
        df[column] = pd.to_numeric(df[column], errors='coerce')
    else:
        print(f"警告：数据框中没有 '{column}' 列")

# 保存修改后的CSV文件
df.to_csv('csv/RAM_products_cleaned.csv', index=False)
print("数据清洗完成，已保存到 csv/RAM_products_cleaned.csv")

In [1]:
import pandas as pd
import re

def parse_product_params(input_file, output_file):
    """
    解析包含产品参数的CSV文件，将参数列拆分为多列

    参数:
        input_file (str): 输入CSV文件路径
        output_file (str): 输出CSV文件路径
    """
    # 读取原始CSV文件
    df = pd.read_csv(input_file)

    # 确保存在'产品参数'列
    if '产品参数' not in df.columns:
        raise ValueError("CSV文件中没有找到'产品参数'列")

    # 预定义常见的参数键（可根据实际数据调整）
    common_params = [
        '适用类型', 'CPU系列', 'CPU主频', '最高睿频', '三级缓存',
        '插槽类型', '核心数量', '线程数',  '热设计功耗(TDP)'
    ]

    # 初始化所有参数列为空字符串
    for param in common_params:
        df[param] = ''

    # 解析每一行的产品参数
    for index, row in df.iterrows():
        if pd.isna(row['产品参数']):
            continue

        # 分割参数行
        param_lines = str(row['产品参数']).split('\n')

        # 解析每个参数键值对
        for line in param_lines:
            # 使用正则表达式匹配键值对
            match = re.match(r'^(.+?)：(.*)$', line.strip())
            if match:
                key = match.group(1).strip()
                value = match.group(2).strip()

                # 如果键在预定义列表中，则添加到DataFrame
                if key in common_params:
                    df.at[index, key] = value
                else:
                    # 对于未预定义的参数，动态添加新列
                    if key not in df.columns:
                        df[key] = ''
                    df.at[index, key] = value

    # 保存处理后的数据到新CSV文件
    df.to_csv(output_file, index=False, encoding='utf_8_sig')
    print(f"处理完成，结果已保存到: {output_file}")

# 使用示例
if __name__ == "__main__":
    input_csv = "csv/测试.csv"  # 替换为你的输入文件路径
    output_csv = "csv/out.csv"  # 输出文件路径

    parse_product_params(input_csv, output_csv)

处理完成，结果已保存到: csv/out.csv


In [2]:
import pandas as pd
import os
import glob

def clean_comment_count(df):
    """清洗评论数据"""
    if 'comment_count' in df.columns:
        # 处理"万+"情况
        df['comment_count'] = df['comment_count'].astype(str).str.replace('万\+', '0000', regex=True)
        # 处理"+"情况
        df['comment_count'] = df['comment_count'].astype(str).str.replace('\+', '', regex=True)
        # 转换为数值类型
        df['comment_count'] = pd.to_numeric(df['comment_count'], errors='coerce')
    return df

def clean_price_columns(df):
    """清洗价格数据"""
    price_columns = ['current_price', 'reference_price']

    for column in price_columns:
        if column in df.columns:
            # 去掉 '￥' 和 '¥' 符号
            df[column] = df[column].astype(str).str.replace('￥', '').str.replace('¥', '')

            # 处理 '万' 字，并转换为数值
            has_wan = df[column].str.contains('万', na=False)
            df.loc[has_wan, column] = (
                df.loc[has_wan, column]
                .str.replace('万', '')
                .astype(float) * 10000
            )

            # 确保所有值转换为数值
            df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

def process_csv_file(file_path):
    """处理单个CSV文件"""
    try:
        # 读取CSV文件
        df = pd.read_csv(file_path)

        # 清洗数据
        df = clean_comment_count(df)
        df = clean_price_columns(df)

        # 保存清洗后的文件
        dir_name = os.path.dirname(file_path)
        file_name = os.path.basename(file_path)
        cleaned_file_name = f"cleaned_{file_name}"
        cleaned_file_path = os.path.join(dir_name, cleaned_file_name)

        df.to_csv(cleaned_file_path, index=False)
        print(f"成功处理并保存: {cleaned_file_path}")

    except Exception as e:
        print(f"处理文件 {file_path} 时出错: {str(e)}")

def batch_process_csv_files(directory='csv'):
    """批量处理目录下的所有CSV文件"""
    # 获取目录下所有CSV文件
    csv_files = glob.glob(os.path.join(directory, '*.csv'))

    if not csv_files:
        print(f"在目录 {directory} 中没有找到CSV文件")
        return

    print(f"找到 {len(csv_files)} 个CSV文件需要处理:")
    for file in csv_files:
        print(f"- {file}")

    # 处理每个文件
    for file_path in csv_files:
        # 跳过已经清洗过的文件(文件名以cleaned_开头)
        if os.path.basename(file_path).startswith('cleaned_'):
            print(f"跳过已处理的文件: {file_path}")
            continue

        process_csv_file(file_path)

    print("所有文件处理完成!")

# 执行批量处理
batch_process_csv_files()

找到 16 个CSV文件需要处理:
- csv\borad_products.csv
- csv\broad_price_history.csv
- csv\chassis_price_history.csv
- csv\chassis_products.csv
- csv\cooler_price_history.csv
- csv\cooler_products.csv
- csv\cpu_price_history.csv
- csv\cpu_products.csv
- csv\gpu_price_history.csv
- csv\gpu_products.csv
- csv\power_price_history.csv
- csv\power_products.csv
- csv\RAM_price_history.csv
- csv\RAM_products.csv
- csv\ssd_price_history.csv
- csv\ssd_products.csv
成功处理并保存: csv\cleaned_borad_products.csv
成功处理并保存: csv\cleaned_broad_price_history.csv
成功处理并保存: csv\cleaned_chassis_price_history.csv
成功处理并保存: csv\cleaned_chassis_products.csv
成功处理并保存: csv\cleaned_cooler_price_history.csv
成功处理并保存: csv\cleaned_cooler_products.csv
成功处理并保存: csv\cleaned_cpu_price_history.csv
成功处理并保存: csv\cleaned_cpu_products.csv
成功处理并保存: csv\cleaned_gpu_price_history.csv
成功处理并保存: csv\cleaned_gpu_products.csv
成功处理并保存: csv\cleaned_power_price_history.csv
成功处理并保存: csv\cleaned_power_products.csv
成功处理并保存: csv\cleaned_RAM_price_history.csv
成